# dev measure birth/death events recursively

In [38]:
from lib.my_initialization import *
from lib.controller.controller_LR import get_one_step_explicit_synchronous_splitting
from lib.model.LR_model import *
from lib.utils.utils_traj import *
from lib.routines.bdrates import *
from lib.measure.utils_measure_tips_cpu import *
from lib.utils.stack_txt_LR import *
from lib.viewer import *
import trackpy
from lib.utils import load_buffer
from lib.routines.return_min_distance import find_stopping_point

#automate the boring stuff
# from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
    
%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## TODO: dev python routine that takes in a spiral tip in pdict and follows it iteratively to its death, zooming in as needed, until a certain absolute tolerance is found in a zero minimum range is reached.  

In [39]:
txt_fn=f'{nb_dir}/Data/test_data/ic008.33_t_218.8.npz'
t=218.8
dt=0.01
txt=load_buffer(txt_fn)
inVc,outVc,inmhjdfx,outmhjdfx,dVcdt=unstack_txt(txt)
width,height=txt.shape[:2]
print(txt.shape)

(200, 200, 18)


In [40]:
V_threshold=-50
jump_threshold = 40
ds=5.0
# get_one_step
__, arr39, one_step = get_one_step_explicit_synchronous_splitting(
    nb_dir,dt,width,height,ds,diffCoef=0.0005,Cm=1.0)

#the heavyweight spiral tip measures
comp_dict_tips=get_comp_dict_topo_full_color(width=width,height=height,level1=V_threshold,level2=0,
                                             jump_threshold=jump_threshold,ds=ds)
# comp_dict_tips=get_compute_all_spiral_tips(width, height, mode='simp')
comp_distance=get_distance_L2_pbc(width,height)

In [42]:
#measure tips that are already present topologically (slower)
img=inVc[...,0];dimgdt=dVcdt[...,0]
# dict_topo=comp_dict_topo_simple(img,dimgdt,t)
dict_topo=comp_dict_tips(img,dimgdt,t,txt)
pid_lst_all=dict_topo['pid']
n_tips=len(pid_lst_all)
n_tips

4

In [46]:
dict_topo.keys()

dict_keys(['x', 'y', 's1', 's2', 't', 'pid', 'greater_pid', 'lesser_pid', 'greater_arclen', 'lesser_arclen', 'greater_arclen_values', 'lesser_arclen_values', 'greater_mean_V', 'lesser_mean_V', 'greater_mean_curvature', 'lesser_mean_curvature', 'greater_xy_values', 'lesser_xy_values', 'greater_V_values', 'lesser_V_values', 'greater_curvature_values', 'lesser_curvature_values'])

In [47]:
#initialize system
pdict=ParticlePBCDict(dict_tips=dict_topo, width=width, height=width)#, **kwargs)
t_prev=t;txt_prev=txt.copy()
# #zoom system
# txt,pdict=zoom_system(txt,pdict)
# txt,pdict=zoom_system(txt,pdict)
# dt=dt/10
# width,height=txt.shape[:2]
# comp_dict_topo_simple=get_comp_dict_topo_simple(width=width,height=height,level1=V_threshold,level2=0)
x_values,y_values,pid_values=pdict.get_current_locations()
inVc,outVc,inmhjdfx,outmhjdfx,dVcdt=unstack_txt(txt)
img=inVc[...,0];dimgdt=dVcdt[...,0]

In [48]:
scale=ds/width
xy_values=np.array(tuple(zip(x_values,y_values)))
assert(xy_values[0].shape[0]==2)

In [49]:
#minimum distance of all pid found
item_lst=[]
for i in range(n_tips):
    for j in range(n_tips):
        if i > j:
            #the ith tip location
            point1=xy_values[i]
            #the jth tip location
            point2=xy_values[j]
            #their distance, in cm
            dist=comp_distance(point1,point2)
            #add to list of distances
            item=(dist,i,j)
            item_lst.append(item)
item_lst=sorted(item_lst)
item_lst

[(0.21228192704422172, 3, 2),
 (19.635486143424185, 1, 0),
 (85.31496830431027, 3, 0),
 (85.49533638494285, 2, 0),
 (96.08268845547843, 2, 1),
 (96.27196815000205, 3, 1)]

In [50]:
#identify the pairs to follow to existential event
num_pairs_to_follow=2
pid_pair_lst=[]
for item in item_lst[:num_pairs_to_follow]:
    d,i,j=item
    pid_pair=i,j
    pid_pair_lst.append(pid_pair)


In [51]:
#define initialization target points as
point_target_init_lst=[]
for pid_pair in pid_pair_lst:
    #the  mean location of these tips 
    point_target_init=np.mean((x_values[pid_pair[0]],x_values[pid_pair[1]])),np.mean((y_values[pid_pair[0]],y_values[pid_pair[1]]))
    point_target_init_lst.append(point_target_init)

In [52]:
#set range to some value
#the ith tip location
point1=np.array(point_target_init_lst[0])
#the jth tip location
point2=np.array(point_target_init_lst[1])
#their distance, in cm
r=comp_distance(point1,point2)*scale #cm distance between the targets
r

2.363878298954001

In [53]:
# zooming in as needed, until a certain absolute tolerance is found in a zero minimum distance between tips is reached.  


:
# consider 
dict_atol={'batch':1e-4,'test':1e-2,'precise':1e-6}
atol=dict_atol['test']
## TODO: turn ^this into a python routine 

In [54]:
# from inspect import getsource
# print(getsource(find_stopping_point))

In [56]:
t_death_lst=[]
for pid_pair in pid_pair_lst:
    #TODO: def follows_pair_to_death(txt,pdict,pid_pair,atol,dt,ds):
    #slow high precision method
    # txt_prev,t_prev,min_sigma_max=find_stopping_point(dt, pdict, txt_prev, t_prev, save_every_n_frames=1)
    #fast low precision method
    txt_prev,t_prev,min_sigma_max=find_stopping_point(dt,pdict,txt_prev,t_prev,save_every_n_frames=1000,V_threshold=V_threshold,pid_pair=pid_pair)
    t_death_lst.append(t_prev+dt)
    
    #init txt
    txt_fn=f'{nb_dir}/Data/test_data/ic008.33_t_218.8.npz'
    t=218.8;dt=0.01
    txt=load_buffer(txt_fn);width,height=txt.shape[:2]
    inVc,outVc,inmhjdfx,outmhjdfx,dVcdt=unstack_txt(txt)
    #initialize system
    dict_topo=comp_dict_tips(img,dimgdt,t,txt)
    #measure tips that are already present topologically (slower)
    img=inVc[...,0];dimgdt=dVcdt[...,0]
    # dict_topo=comp_dict_topo_simple(img,dimgdt,t)
    dict_topo=comp_dict_tips(img,dimgdt,t,txt)
    pid_lst_all=dict_topo['pid']
    n_tips=len(pid_lst_all)
    # #zoom system
    # txt,pdict=zoom_system(txt,pdict)
    # txt,pdict=zoom_system(txt,pdict)
    # width,height=txt.shape[:2]
    # comp_dict_topo_simple=get_comp_dict_topo_simple(width=width,height=height,level1=V_threshold,level2=0)
    # x_values,y_values,pid_values=pdict.get_current_locations()
    pdict=ParticlePBCDict(dict_tips=dict_topo, width=width, height=width)#, **kwargs)
    t_prev=t;txt_prev=txt.copy()

death event found for pid_pair=(3, 2) at time t=228.79999999999092, where dt=0.01 and L=200...
	 min_sigma_max=0.01397989604501346 cm
death event found for pid_pair=(1, 0) at time t=248.79999999997273, where dt=0.01 and L=200...
	 min_sigma_max=0.014290084392902717 cm


In [57]:
#TODO: determine which pid_pair dies first
# item=t_death,pid_pair
sorted_pid_pair_id_lst=np.argsort(t_death_lst)
# boo is whether the last pid_pair died last)
# boo=t_death_lst[-1]==np.array(t_death_lst)
#for testing only 
boo=True

In [58]:
#TODO(if the last pid_pair didn't die last): reinititialize txt
if not boo:
    txt_fn=f'{nb_dir}/Data/test_data/ic008.33_t_218.8.npz'
    t=218.8
    dt=0.01
    txt=load_buffer(txt_fn)
    inVc,outVc,inmhjdfx,outmhjdfx,dVcdt=unstack_txt(txt)
    width,height=txt.shape[:2]
    print(txt.shape)
    #etc.

In [59]:
sorted_pid_pair_id_lst=[0,1]
print(f"t={t:0.2f}")
#DONE: enforce sorting of tips at a low level that integrates with all pythonic methods

t=218.80


In [35]:
#TODO: init txt
txt_fn=f'{nb_dir}/Data/test_data/ic008.33_t_218.8.npz'
t=218.8
dt=0.01
txt=load_buffer(txt_fn)
inVc,outVc,inmhjdfx,outmhjdfx,dVcdt=unstack_txt(txt)
width,height=txt.shape[:2]
print(txt.shape)

#TODO: recursively zoom into the first/given death up to atol.  return pdict upto that point

#TODO: repeat while there are still pid_pairs to zoom into

#TODO(laterish): dev merge_with_other method to merge pdict1 and pdict2

(200, 200, 18)


In [60]:
# #TODO: test measure_kernel
# retval=find_tips_with_pbc_knots_full_color(contours1, contours2, s1in_lst, s2in_lst, txt)
# s1_list, s2_list, x_lst, y_lst, txt_lst=retval
# #TODO: call ^this function with contours_to_simple_tips_pbc_full_color

# TODO: test/dev recursive_zoom_to_death
then,<br>
TODO: define recursive_zoom_to_birth

In [83]:
# TODO: debug deaths
def recursive_zoom_to_death(pid_pair_lst,txt_in,dt,V_threshold=-50.,atol=1e-2,
                            one_steps_per_frame=100,ds=5.,
                            **kwargs):
    retval_lst=[]
    for pid_pair in pid_pair_lst:
        txt_prev=txt_in.copy()
        # one_step,comp_distance,comp_dict_tips=init_methods(txt_in,ds,dt,V_threshold=-50,jump_threshold=40)

        txt_prev,t_prev,min_sigma_max=find_stopping_point(dt,pdict,txt_prev,t_prev,
                                                          save_every_n_frames=one_steps_per_frame,
                                                          V_threshold=V_threshold,
                                                          pid_pair=pid_pair,ds=ds)
        t_death=t_prev+dt
        system=list(zip(t_death,pid_pair,txt_prev))
        boo=check_atol(pdict,pid_pair,atol)
        while boo:
            system=zoom_system(system)
            t_death,pid_pair,txt_prev=system
            dt/=10.
            #TODO: compute each of pdict,txt_prev, t_prev=t_death-dt,V_threshold, 
            t_death=find_stopping_point(dt,pdict,txt_prev,t_prev,save_every_n_frames=10,V_threshold=V_threshold,pid_pair=pid_pair,ds=ds)
            boo=check_atol(system,atol)
        retval = t_death,min_sigma_max
        retval_lst.append(retval)
    return retval_lst
    
# TODO: and then births
def recursive_zoom_to_birth(pid_pair_lst,in_fn,ds,dt,**kwargs):
    '''TODO(later)'''
    retval_lst=[]
    for pid_pair in pid_pair_lst:
        retval=init_system(in_fn,ds,dt,**kwargs)
        t_birth=find_starting_point(pdict,pid_pair,**kwargs)
        #TODO: after finding starting point, 
        # - integrate forward n_steps
        # - return apparent starting point {and return pdict}
        # - reinit at txt_prev / t_min, where t_min is the min(pdict.times)          
        #TODO: factor in the tscale/sscale
        #TODO:  find whether to zoom
        while boo:
            system=zoom_system(system)
            t_birth=find_starting_point(pdict,pid_pair,**kwargs)
            boo=check_atol(system,atol)
        retval = t_birth
        retval_lst.append(retval)
    return retval_lst

In [90]:
def check_atol(pdict,pid_pair,atol,index=-1,ds=5.):
    particle1=pdict[pid_pair[0]]
    particle2=pdict[pid_pair[1]]
    point=np.array((particle1['x'][index],particle1['y'][index]))
    scale=ds/particle1.width
    dist=particle2.dist_to(point)*scale
    boo=dist<atol
    return boo

In [91]:
txt_in=load_buffer(txt_fn)

retval_lst=recursive_zoom_to_death(pid_pair_lst,txt_in,dt,V_threshold=-50.,atol=1e-2,
                            one_steps_per_frame=100,ds=5.)

TypeError: init_system() missing 3 required positional arguments: 'txt_in', 'ds', and 'dt'

In [89]:
# def find_starting_point(pdict,pid_pair,**kwargs):
#     #TODO: follow pdict back to the birth of this pid_pair.  Supposes pid_pair denotes a birth
#     #TODO(don't): modify find_stopping point
#     pass
def init_methods(txt_in,ds,dt,V_threshold=-50,jump_threshold=40,**kwargs):
    '''init_system(txt_in,ds,dt,V_threshold=-50,jump_threshold = 40'''
    width,height=txt_in.shape[:2]
    #TODO: pass kwargs to the get_one_step in use
    # get_one_step
    __, arr39, one_step = get_one_step_explicit_synchronous_splitting(
        nb_dir,dt,width,height,ds,diffCoef=0.0005,Cm=1.0)

    #the heavyweight spiral tip measures
    comp_dict_tips=get_comp_dict_topo_full_color(width=width,height=height,level1=V_threshold,level2=0,
                                                 jump_threshold=jump_threshold,ds=ds)
    # comp_dict_tips=get_compute_all_spiral_tips(width, height, mode='simp')
    comp_distance=get_distance_L2_pbc(width,height)
    
    
    one_step=get_one_step(width,height)
    return one_step,comp_distance,comp_dict_tips

def zoom_in(txt,pdict,dt,level1,level2,tfactor=0.1):
    '''
    Example Usage:
    txt, pdict, comp_tips, one_step=zoom_in(txt,pdict,dt,level1,level2,tfactor=0.1)
    '''
    # zooming in as needed, until a certain absolute tolerance is found in a zero minimum distance between tips is reached.  
    dt=tfactor*dt
    width,height=txt.shape[:2]
    comp_tips=get_comp_dict_topo_full_color(width=width,height=height,level1=level1,level2=level2)
    # get_one_step at this dt (Luo-Rudy model)
    __, arr39, one_step = get_one_step_explicit_synchronous_splitting(
        nb_dir,dt=dt,width=width,height=height,ds=ds,diffCoef=0.0005,Cm=1.0)
    
    txt,pdict=zoom_system(txt,pdict)
    return txt, pdict, dt, comp_tips, one_step

def zoom_system(system, dt, pdict):
    zoom_in(txt,pdict,dt,level1,level2,tfactor=0.1)
    for systm in system:
        t_prev,pid_pair,txt_prev=systm
        txt, pdict, comp_tips, one_step=zoom_in(txt,pdict,dt,level1=-50,level2=0.,tfactor=0.1)
        #TODO: zoom_system (HINT: ctrl+F for zoom_ in lib)
    system=txt, pdict, dt, comp_tips, one_step

In [ ]:
txt=pdict.record_tips_return_txt(txt,duration,one_step,comp_tips,dt,save_every_n_frames=1)
